In [ ]:
!git clone https://github.com/fastai/fastai1.git

Cloning into 'fastai1'...
remote: Enumerating objects: 32976, done.
remote: Counting objects: 100% (207/207), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 32976 (delta 95), reused 197 (delta 91), pack-reused 32769
Receiving objects: 100% (32976/32976), 471.68 MiB | 28.22 MiB/s, done.
Resolving deltas: 100% (23992/23992), done.


In [ ]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 36.8 MB 1.3 MB/s 


In [ ]:
import sys
import os
import requests
import subprocess
import shutil
from logging import getLogger, StreamHandler, INFO

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*') # switch off RDKit warning messages

from sklearn.model_selection import train_test_split

from fastai1.fastai import *
from fastai1.fastai.text import *
from fastai1.fastai.vision import *

import numpy as np
import threading
import random
from sklearn.utils import shuffle 

In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Create a path to save the results
from pathlib import Path

data_path = Path('/content/gdrive/MyDrive/AI /Drug Discovery/TL_Catalysis_Code/results')
name = 'regressor_30 '
path = data_path/name
path.mkdir(exist_ok=True, parents=True)

In [ ]:
import pandas as pd

data = pd.read_csv('/content/gdrive/MyDrive/AI /Drug Discovery/TL_Catalysis_Code/Data/Fine-Tuning/reaction-3.csv')
print('Dataset:', data.shape)

Dataset: (368, 2)


In [ ]:
random_seed(1234, True)

train_ , test = train_test_split(data, test_size=0.20, random_state=100)
train, valid = train_test_split(train_, test_size=0.125, random_state=0)
print(train.shape)
print(test.shape)
print(valid.shape)

(257, 2)
(74, 2)
(37, 2)


In [ ]:
def randomize_smiles(smiles):
    m = Chem.MolFromSmiles(smiles)
    ans = list(range(m.GetNumAtoms()))
    np.random.shuffle(ans)
    nm = Chem.RenumberAtoms(m,ans)
    return Chem.MolToSmiles(nm, canonical=False, isomericSmiles=True, kekuleSmiles=False)

In [ ]:
# For Reaction-3 'yield' is replaced by 'ee'.
def ee_smiles_augmentation(df, N_rounds, noise):
    '''
    noise: add gaussion noise to the label
    '''
    dist_aug = {col_name: [] for col_name in df}

    for i in range(df.shape[0]):
        for j in range(N_rounds):
            dist_aug['smiles'].append(randomize_smiles(df.iloc[i].smiles))
            dist_aug['ee'].append(df.iloc[i]['ee'] + np.random.normal(0,noise))
    df_aug = pd.DataFrame.from_dict(dist_aug)
    df_aug = df_aug.append(df, ignore_index=True)
    return df_aug.drop_duplicates('smiles')

In [ ]:
%%time 
random_seed(1234, True)

# N_rounds is changed from 100 to 50
train_aug = ee_smiles_augmentation(train, 100, noise=0.5)
print("Train_aug: ", train_aug.shape)

Train_aug:  (25950, 2)
CPU times: user 24.4 s, sys: 53.3 ms, total: 24.5 s
Wall time: 24.5 s


In [ ]:
# Don't include the defalut specific token of fastai, only keep the padding token
BOS,EOS,FLD,UNK,PAD = 'xxbos','xxeos','xxfld','xxunk','xxpad'
TK_MAJ,TK_UP,TK_REP,TK_WREP = 'xxmaj','xxup','xxrep','xxwrep'
defaults.text_spec_tok = [PAD]

special_tokens = ['[BOS]', '[C@H]', '[C@@H]','[C@]', '[C@@]','[C-]','[C+]', '[c-]', '[c+]','[cH-]',
                   '[nH]', '[N+]', '[N-]', '[n+]', '[n-]' '[NH+]', '[NH2+]',
                   '[O-]', '[S+]', '[s+]', '[S-]', '[O+]', '[SH]', '[B-]','[BH2-]', '[BH3-]','[b-]',
                   '[PH]','[P+]', '[I+]', 
                  '[Si]','[SiH2]', '[Se]','[SeH]', '[se]', '[Se+]', '[se+]','[te]','[te+]', '[Te]']

class MolTokenizer(BaseTokenizer):
    def __init__(self, lang = 'en', special_tokens = special_tokens):
        self.lang = lang
        self.special_tokens = special_tokens
        
    def tokenizer(self, smiles):
        # add specific token '[BOS]' to represetences the start of SMILES
        smiles = '[BOS]' + smiles
        regex = '(\[[^\[\]]{1,10}\])'
        char_list = re.split(regex, smiles)
        tokens = []
        
        if self.special_tokens:
            for char in char_list:
                if char.startswith('['):
                    if char in special_tokens:
                        tokens.append(str(char))
                    else:
                        tokens.append('[UNK]')
                else:
                    chars = [unit for unit in char]
                    [tokens.append(i) for i in chars]                    
        
        if not self.special_tokens:
            for char in char_list:
                if char.startswith('['):
                    tokens.append(str(char))
                else:
                    chars = [unit for unit in char]
                    [tokens.append(i) for i in chars]
                
        #fix the 'Br' be splited into 'B' and 'r'
        if 'B' in tokens:
            for index, tok in enumerate(tokens):
                if tok == 'B':
                    if index < len(tokens)-1: # make sure 'B' is not the last character
                        if tokens[index+1] == 'r':
                            tokens[index: index+2] = [reduce(lambda i, j: i + j, tokens[index : index+2])]
        
        #fix the 'Cl' be splited into 'C' and 'l'
        if 'l' in tokens:
            for index, tok in enumerate(tokens):
                if tok == 'l':
                    if tokens[index-1] == 'C':
                            tokens[index-1: index+1] = [reduce(lambda i, j: i + j, tokens[index-1 : index+1])]
        return tokens    
    
    def add_special_cases(self, toks):
        pass

In [ ]:
bs = 128
tok = Tokenizer(partial(MolTokenizer, special_tokens = special_tokens), n_cpus=6, pre_rules=[], post_rules=[])

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

# Here also 'yield' is replaced with 'ee'. 

random_seed(1234, True)
data_clas = TextClasDataBunch.from_df(path, train_aug, valid, bs=bs, tokenizer=tok, 
                                          chunksize=50000, text_cols='smiles',label_cols='ee', 
                                          max_vocab=60000, include_bos=False, min_freq=1, num_workers=0)

print(f'Vocab Size: {len(data_clas.vocab.itos)}')

Vocab Size: 40


In [ ]:
random_seed(1234, True)

reg_learner = text_classifier_learner(data_clas, AWD_LSTM, pretrained=False, drop_mult=0.0, metrics = [rmse, r2_score])
reg_learner.unfreeze()

In [ ]:
# Model architecture
reg_learner.model

SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(40, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(40, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): Sequential(
      (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=1200, out_features=50, bias=True)
      (2): ReLU(inplace=True)
      (3): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_run

In [ ]:
# To turn off the warning in "VisibleDeprecationwarning"
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

random_seed(1234, True)

lr = 1e-3

# I have changed it to one epoch instead of 15 epoch. 
reg_learner.fit_one_cycle(8, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,root_mean_squared_error,r2_score,time
0,7633.459961,7842.604004,88.558479,-67.178238,01:14
1,7169.132324,3763.865723,61.350353,-31.720478,01:10
2,5934.380371,15355.312500,123.916557,-132.488602,01:07
3,4254.296875,3459.701904,58.819229,-29.076284,01:10
4,2969.098877,2131.375488,46.166821,-17.528721,01:09
5,2129.864014,4876.424805,69.831406,-41.392307,01:08
6,1757.316528,1944.636230,44.098030,-15.905337,01:07
7,1705.057739,1872.971924,43.277847,-15.282339,01:11


In [ ]:
split_id = 30   # Model id for reaction 3 & m0
reg_learner.save(f'{split_id}_reg')

In [ ]:
def test_smiles_augmentation(df, N_rounds):
    dist_aug = {col_name: [] for col_name in df}

    for i in range(df.shape[0]):
        for j in range(N_rounds):
            dist_aug['smiles'].append(randomize_smiles(df.iloc[i].smiles))
            dist_aug['ee'].append(df.iloc[i]['ee'])
    df_aug = pd.DataFrame.from_dict(dist_aug)
    
    return pd.DataFrame.from_dict(dist_aug)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

preds = []

# Randomized SMILES Predictions
for i in range(4):
    np.random.seed(12*i)

    test_aug = test_smiles_augmentation(test,1)

    # above, I have changed test_smiles_augmentation to smiles augmentation. 

    #model (yeild changed to ee)
    test_db = TextClasDataBunch.from_df(path, train, test_aug, tokenizer=tok, vocab=data_clas.vocab,
                                            text_cols='smiles', label_cols='ee', bs=bs, include_bos=False)
    learner = text_classifier_learner(test_db, AWD_LSTM, pretrained=False, drop_mult=0.0, metrics = [r2_score, rmse])
    
    learner.load(f'{split_id}_reg'); 
  
    #get predictions
    pred,lbl = learner.get_preds(ordered=True)
    preds.append(pred)
    
# Canonical SMILES Predictions

# Here 'yield' is replaced by 'ee'.

test_db = TextClasDataBunch.from_df(path, train, test, bs=bs, tokenizer=tok, 
                              chunksize=50000, text_cols='smiles',label_cols='ee', vocab=data_clas.vocab, max_vocab=60000,
                                              include_bos=False)

learner = text_classifier_learner(test_db, AWD_LSTM, pretrained=False, drop_mult=0.0, metrics = [r2_score, rmse])

learner.load(f'{split_id}_reg');

#get predictions
pred_canonical,lbl2 = learner.get_preds(ordered=True)

# i have commented this line to avoid size related error in tensor.     
#preds.append(pred_canonical)
#preds

In [ ]:
print('Test Set (Canonical)')
print('RMSE:', root_mean_squared_error(pred_canonical,lbl2))
print('R2:', r2_score(pred_canonical,lbl2))
print('MAE:', mean_absolute_error(pred_canonical,lbl2))

avg_preds = sum(preds)/len(preds)
#print('\n')
print('Test Set (Average)')
print('RMSE:', root_mean_squared_error(avg_preds,lbl))
print('R2:', r2_score(avg_preds,lbl))
print('MAE:', mean_absolute_error(avg_preds,lbl))

Test Set (Canonical)
RMSE: tensor(42.0245)
R2: tensor(-3.5397)
MAE: tensor(40.3457)
Test Set (Average)
RMSE: tensor(47.5644)
R2: tensor(-4.8155)
MAE: tensor(46.3364)
